# Squin gates of interest

given immutable lists `q: IList[Qubit, Literal[7]]` of qubits, and `p: float` noise rate

### Depolarizing noise channels

In [16]:
from bloqade import squin
from kirin.dialects.ilist import IList


# a squin kernel
@squin.kernel
def good_to_know(p: float = 0):
    q = squin.qalloc(3)  # allocate qubits

    # Single qubit noise with parameter p; broadcast applies channel in parallel to all qubits in list
    squin.broadcast.depolarize(p, IList([q[0], q[1]]))

    # Two-qubit noise
    squin.depolarize2(p, q[0], q[1])

### Measure and reset

In [17]:
# measure and reset example
@squin.kernel
def measure_reset_example():
    q = squin.qalloc(3)  # allocate qubits first
    
    # measure (broadcast applies to all qubits in register)
    squin.broadcast.measure(q)
    
    # reset (broadcast applies to all qubits in register)
    squin.broadcast.reset(q)

note that to implement resets and measurements on single qubits, just drop `.broadcast` and apply to single-qubit registers.

# From Bloqade Squin kernels to stabilizer samplers
### Convert circuit to Stim/Tsim, visualize, run sampler

In [18]:
import bloqade.stim
import bloqade.tsim
import numpy as np


# Magic state injection circuit for [[7,1,3]] color code
# Creates logical observable qubit |ψ_L(θ,φ)⟩
@squin.kernel
def main(theta: float = 0.0, phi: float = 0.0):
    # Allocate 7 qubits (q[0] to q[6])
    q = squin.qalloc(7)
    
    # Qubits 0-5 initialized in |0⟩ (default)
    # Qubit 6 initialized in |ψ(θ,φ)⟩
    squin.h(q[6])  # Prepare |+⟩
    squin.ry(angle=theta, qubit=q[6])  # Rotate by theta
    squin.rz(angle=phi, qubit=q[6])  # Phase rotation
    
    # Step 1: Apply √Y† gates to qubits 0-5
    # √Y† = RY(-π/2) ≈ -1.5708
    for i in range(6):
        squin.ry(angle=-1.5707963267948966, qubit=q[i])  # -π/2
    
    # Step 2: CNOT gates (following the circuit diagram)
    squin.cx(control=q[1], target=q[2])  # Wire 1 controls wire 2
    squin.cx(control=q[3], target=q[4])  # Wire 3 controls wire 4
    squin.cx(control=q[5], target=q[6])  # Wire 5 controls wire 6
    squin.cx(control=q[0], target=q[2])  # Wire 0 controls wire 2
    squin.cx(control=q[2], target=q[4])  # Wire 2 controls wire 4
    squin.cx(control=q[4], target=q[6])  # Wire 4 controls wire 6
    squin.cx(control=q[6], target=q[4])  # Wire 6 controls wire 4
    
    # Step 3: Apply √Y gates to qubits 1-6
    # √Y = RY(π/2) ≈ 1.5708
    for i in range(1, 7):
        squin.ry(angle=1.5707963267948966, qubit=q[i])  # π/2


# Print circuit representation
print("Circuit structure for logical observable |ψ_L(θ,φ)⟩:")
print("  - 7 qubits (q[0] to q[6])")
print("  - Qubits 0-5: initialized in |0⟩")
print("  - Qubit 6: initialized in |ψ(θ,φ)⟩")
print("  - √Y† gates on qubits 0-5")
print("  - CNOT gates: 1→2, 3→4, 5→6, 0→2, 2→4, 4→6, 6→4")
print("  - √Y gates on qubits 1-6")
print("  - Output: Logical observable |ψ_L(θ,φ)⟩")

# Plotting circuit diagram (plotting is nicer on Tsim)
print("\nCreating Tsim circuit for visualization...")
tsim_circ = bloqade.tsim.Circuit(main)
# Display the circuit diagram
tsim_circ.diagram(height=400)

# Sampling using Stim (Clifford-only sampling is faster on Stim)
# Note: For sampling, we need measurements
@squin.kernel
def main_with_measurements(theta: float = 0.0, phi: float = 0.0):
    q = squin.qalloc(7)
    squin.h(q[6])
    squin.ry(angle=theta, qubit=q[6])
    squin.rz(angle=phi, qubit=q[6])
    for i in range(6):
        squin.ry(angle=-1.5707963267948966, qubit=q[i])  # -π/2
    squin.cx(control=q[1], target=q[2])
    squin.cx(control=q[3], target=q[4])
    squin.cx(control=q[5], target=q[6])
    squin.cx(control=q[0], target=q[2])
    squin.cx(control=q[2], target=q[4])
    squin.cx(control=q[4], target=q[6])
    squin.cx(control=q[6], target=q[4])
    for i in range(1, 7):
        squin.ry(angle=1.5707963267948966, qubit=q[i])  # π/2
    squin.broadcast.measure(q)  # Measure all qubits for sampling

print("\nCreating Stim circuit for sampling...")
stim_circ = bloqade.stim.Circuit(main_with_measurements)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=100)
print(f"✓ Sampled {len(samples)} shots successfully!")
print(f"  Sample shape: {samples.shape} (shots × qubits)")

Circuit structure for logical observable |ψ_L(θ,φ)⟩:
  - 7 qubits (q[0] to q[6])
  - Qubits 0-5: initialized in |0⟩
  - Qubit 6: initialized in |ψ(θ,φ)⟩
  - √Y† gates on qubits 0-5
  - CNOT gates: 1→2, 3→4, 5→6, 0→2, 2→4, 4→6, 6→4
  - √Y gates on qubits 1-6
  - Output: Logical observable |ψ_L(θ,φ)⟩

Creating Tsim circuit for visualization...


InterpreterError: Interpreter EmitStimMain does not support Div() using py.binop dialect. Expected lowering.func, stim.noise, stim.gate, ssacfg, lowering.call, debug, func, stim.collapse, stim.aux

# On noise 
### Using and adjusting heuristic noise model

In [ ]:
from bloqade.cirq_utils import noise
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import load_circuit


# applying noise model to circuit according to different architectures; must use Cirq circuit
@squin.kernel
def main():
    my_kernel()


cirq_main = emit_circuit(main)  # emit to Cirq
noise_model = (
    noise.GeminiOneZoneNoiseModel()
)  # define noise model; here OneZone default
main_noisy = noise.transform_circuit(
    cirq_main, model=noise_model
)  # annotate circuit w noise
squin_main = load_circuit(main_noisy)  # back to Squin
noisy_stim = bloqade.stim.Circuit(squin_main)  # now to Stim


# Example 1: defining new noise model on top of OneZone architecture, scaling all noise parameters by a constant factor
noise_scale = 1
noise_model_1 = noise.GeminiOneZoneNoiseModel(scaling_factor=noise_scale)

# Example 2: changing just some of the channels: no local phase-z errors
noise_model_2 = noise.GeminiOneZoneNoiseModel(local_pz=0.0)